In [ ]:
!pip install --quiet tensorflow-text
# Clone the entire repo.
%cd /content/
!rm -r -f dl4tm
!git clone git://github.com/Jeilef/DL4TM_Text2Python.git dl4tm
%cd dl4tm
!ls

In [7]:
import numpy as np
import json
import logging
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text # See https://github.com/tensorflow/hub/issues/463

In [8]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [2]:
train_file = open("conala-corpus-v1.1/conala-corpus/conala-train.json", "r")
train_json = json.load(train_file)

print(len(train_json))
print(train_json[0])

2379
{'intent': 'How to convert a list of multiple integers into a single integer?', 'rewritten_intent': "Concatenate elements of a list 'x' of multiple integers to a single integer", 'snippet': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))', 'question_id': 41067960}


In [3]:
base_sentences = [item["intent"] for item in train_json]
codes = [item["snippet"] for item in train_json]

print(base_sentences[0])
print(codes[0])

How to convert a list of multiple integers into a single integer?
sum(d * 10 ** i for i, d in enumerate(x[::-1]))


In [17]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

embedding_model = tf.keras.Model(text_input, sequence_output)
sentences = tf.constant(["Hello World!", "How are you?", "(your text here)"])
embedding = embedding_model(sentences)
# tf.keras.utils.plot_model(embedding_model)
print(embedding)

tf.Tensor(
[[[-1.3101449   0.02224976 -2.4941747  ... -0.9011016  -1.2354522
   -0.47412774]
  [-2.1429274  -0.37572432 -0.42352897 ... -2.256411   -2.7875555
    0.11143181]
  [-1.5082821  -0.432453   -0.30818245 ... -1.7291834  -0.9929765
   -0.16822301]
  ...
  [-1.224653   -0.35186255 -0.24392751 ... -1.4310317  -1.0778803
    0.92555493]
  [-1.1960835   0.00612556 -0.3530901  ... -1.0848111  -1.195029
    0.92324764]
  [-0.977689    0.05358659 -0.50089896 ... -0.9360498  -1.302382
    0.7090657 ]]

 [[-1.1281638   0.4898095  -2.6764565  ... -0.12474971  0.25346345
   -0.33696735]
  [-1.3290354   0.4153836  -0.8495089  ... -0.2951286  -0.15117727
    0.07448988]
  [-1.2831266  -0.00796435  0.22752291 ... -1.0752413  -0.26373333
    0.05724901]
  ...
  [-1.2007624  -0.47225767  0.05070373 ... -1.0925945   0.71441954
    0.86031246]
  [-0.993935    0.12878755 -0.16948964 ... -1.0883722   0.25889325
    1.1489222 ]
  [-1.0162177   0.6680834  -0.2553135  ... -0.91790366 -0.13104461
   

In [18]:
sample_decoder = Decoder(num_layers=2, d_model=128, num_heads=2,
                         dff=128, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((3, 26), dtype=tf.int64, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=embedding,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

(TensorShape([3, 26, 128]), TensorShape([3, 2, 26, 128]))